<a href="https://colab.research.google.com/github/mvdheram/Stereotypical-Social-bias-detection-/blob/Pre-trained-LM-selection-and-training/Pre_trained_Language_model_selection_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Selecting pre-trained language model- Research

# Selected models notes - Research

# Training selected LM - Implementation 

# Analyzing results - Research